# A/B Test

## Project Description
You've received an analytical task from an international online store. Your predecessor failed to complete it: they launched an A/B test and then quit. They left only the technical specifications and the test results.

## Technical description
* Test name: recommender_system_test
* Groups: А (control), B (new payment funnel)
* Launch date: 2020-12-07
* The date when they stopped taking up new users: 2020-12-21
* End date: 2021-01-01
* Audience: 15% of the new users from the EU region
* Purpose of the test: testing changes related to the introduction of an improved recommendation system
* Expected result: within 14 days of signing up, users will show better conversion into product page views (the product_page event), product card views (product_card) and purchases (purchase). At each of the stage of the funnel product_page → product_card → purchase, there will be at least a 10% increase.
* Expected number of test participants: 6000



## 1. Load and Analyze Files

In [3]:
# load libraries
import pandas as pd
import numpy as np
import datetime as dt
import math
from scipy import stats as st
from matplotlib import pyplot as plt
import seaborn as sns
from plotly import graph_objects as go
from statsmodels.stats.proportion import proportions_ztest
import warnings
warnings.filterwarnings("ignore")


In [4]:
# load files
marketing_events = pd.read_csv('/datasets/ab_project_marketing_events_us.csv', parse_dates = ['start_dt', 'finish_dt'])
new_users = pd.read_csv('/datasets/final_ab_new_users_upd_us.csv', parse_dates = ['first_date'])
events = pd.read_csv('/datasets/final_ab_events_upd_us.csv', parse_dates = ['event_dt'])
participants = pd.read_csv('/datasets/final_ab_participants_upd_us.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/ab_project_marketing_events_us.csv'

In [ ]:
files =  [marketing_events, new_users, events, participants]
for file in files:
    display(file.head())
    display(file.info())
    print("\n")

## 2. Data preprocessing


 We had already y changed the data type for all date columns to datetime. We did that for marketing_events, new_users, and events tables. So, first, we will check for ducplicate rows in all tables

In [ ]:
file.duplicated().sum()
for file in files:
    display(file.duplicated().sum())

There are no duplicate rows.  From the previous step, we can see that values are missing in the details column in the events table.

In [ ]:
events.groupby['events_name']('details').unique()

Other than 'purcharse', we don't have information for the other events. Nothing seems wrong.

Analyzing descriptive info for 'new users' and 'events'

In [ ]:
display(new_users)['first_date'].describe()
events['events_dt'].describe()

There are new users later than the specified date of 2020-12-21, up to 2020-12-23. We can see that the last event date is 2020-12-30, compared to the specified date - 2021-01-01. Let's take a look on the ab_test values from participants table:

In [ ]:
participants['ab_test'].unique()

There's two tests. Since we need to study 'interface_eu_test', we'll drop the drop participants of the 'recommender_system_test'

In [ ]:

participants = participants[participants['ab_test'] == 'interface_eu_test']

# Drop the 'ab_test' column as it's no longer needed
participants = participants.drop(columns=['ab_test'])


print("\nFiltered participants DataFrame:")
participants

Verifying if we have users in both groups as well as the amount of participants for each group

In [ ]:
#Group by user_id and count unique groups
grouped = participants.groupby('user_id')['group'].nunique()

#Filter users who are in more than one group
filtered = grouped[grouped > 1]

#Get the number of such users
num_users_in_multiple_groups = len(filtered)

num_users_in_multiple_groups

There are no users assigned to both groups. We have similar number of participants in both group, and in total much more than the 6000 required. Now We'll merge the dataframes to one:

In [ ]:
final_df = events.merge (paprticipants, on = 'user_id', how = 'inner')
final_df = final_df.merge(new_users, on='user_id', how='inner')
final_df.head()
final_df.shape


In [ ]:
final_df.groupby('group')['user_id'].nunique()

## 3. Exploratory Data Analysis (EDA)

Checking the logs to see which events occurred and how often they occurred

In [ ]:
final_df['event_name'].value_counts()

Let's calculate  the number of unique users who performed each event and the proportion of these users relative to the total number of unique users.

In [ ]:
events_funnel = final_df.groupby('event_name')['user_id'].nunique().reset_index()
events_funnel.columns = ['events', 'users']
total_users = final_df['user_id'].nunique()
events_funnel['share_%'] =  (events_funnel['users'] / total_users * 100).round(2)
events_funnel = events_funnel.sort_values(by='users', ascending=False).reset_index(drop=True)
events_funnel

From the technical description, the event funnel is login > product page > product_cart (optional) > purchase. We'll need to change the order of purchase and product_cart and get the visual funnel without the product_cart event:

In [ ]:
events_funnel = events_funnel.set_index(pd.Index([0, 1, 3, 2]))
events_funnel.sort_index(inplace=True)
events_funnel

In [ ]:
fig = go.Figure(go.Funnel(
    y = events_funnel['event'],
    x = events_funnel['users'], textinfo = "value+percent previous"))
fig.update_layout(title='Sales funnel for experimental groups')
fig.show()

In [5]:


def create_funnel_data(df, group=None):
    if group:
        df = df[df['group'] == group]
    funnel_data = df.groupby('event_name')['user_id'].nunique().reset_index()
    funnel_data.columns = ['events', 'users']
    total_users = df['user_id'].nunique()
    funnel_data['share_%'] = (funnel_data['users'] / total_users * 100).round(2)

    if len(funnel_data) >= 4:
        funnel_data = funnel_data.set_index(pd.Index([0, 1, 3, 2]))
        funnel_data.sort_index(inplace=True)
    else:
        funnel_data = funnel_data.reset_index(drop=True)

    return funnel_data

# Assuming final_df is already defined
# Create funnel data for all groups
events_funnel = create_funnel_data(final_df)

# Create funnel data for group A
events_funnel_a = create_funnel_data(final_df, group='A')

# Create funnel data for group B
events_funnel_b = create_funnel_data(final_df, group='B')

# Display the funnel data for group A and B to verify
print(events_funnel_a)
print(events_funnel_b)

# Plot the funnel for both groups
fig = go.Figure()

fig.add_trace(go.Funnel(
    name='Group A',
    y=events_funnel_a['events'],
    x=events_funnel_a['users'], 
    textinfo="value+percent previous"))
fig.add_trace(go.Funnel(
    name='Group B',
    y=events_funnel_b['events'],
    x=events_funnel_b['users'], 
    textinfo="value+percent previous"))

fig.update_layout(title='Sales funnel for experimental groups')
fig.show()




NameError: name 'final_df' is not defined

**COMMENT ON GRAPHS**

Is the number of events per user distributed equally in the samples?

In [ ]:

# Group by 'group' and calculate the number of unique users and total events
events_group_per_user = final_df.groupby('group').agg(
    num_users=('user_id', 'nunique'), 
    total_events=('event_dt', 'count')
).reset_index()

# Calculate the average number of events per user
events_group_per_user['average_events_per_user'] = events_group_per_user['total_events'] / events_group_per_user['num_users']


events_group_per_user

Let's check the relative frequency of each event within each group:

In [ ]:
# Group by 'group' and count the value occurrences of 'event_name', normalize to get proportions
event_group = final_df.groupby('group')['event_name'].value_counts(normalize=True).reset_index(name='share')
event_group['share'] = (event_group['share'] * 100).round(2)

event_group

In [ ]:
ax = sns.barplot(x="event_name", y="share", hue="group", data=event_group).set_title("Event Distribution")
plt.show()

We can confirm that the number of events per user distributed equally in the samples. Now we will check for users who have been assigned to multiple groups in an A/B test,  count the number of users who are present in exactly one group within 'final_df'

In [ ]:
final_df.groupby(["user_id"])["group"].nunique().reset_index().query("group>1").shape[0]


In [ ]:
final_df.groupby(["user_id"])["group"].nunique().reset_index().query("group==1").shape[0]


How is the number of events distributed by days?

In [ ]:
final_df['event_date']= final_df['event_dt'].dt.date
days = final_df.groupby(['event_date']).agg({'user_id':'nunique', 'event_dt':'count'}).reset_index()

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Number of events distribution among days')
sns.lineplot(data=days, x='event_date', y='event_dt', label='Events')
plt.xlabel('Date')
plt.xticks(days.event_date, rotation=90)
plt.ylabel('Events')
plt.show()

**COMMENT ON GRAPH**

In [ ]:
final_df[['event_date']] = final_df['event_dt'].dt.date
days = final_df.groupby(['event_date', 'group']).agg({'user_id':'nunique', 'event_dt':'count'}).reset_index()
plt.figure(figsize=(16,8))
plt.title('Number of events distribution among days')
sns.lineplot(data=days, x='event_date', y='event_dt', hue="group")
plt.xlabel('Date')
plt.xticks(days.event_date, rotation=90)
plt.ylabel('Events')
plt.show()

The distribution of events per test group is very similar.

## 4. Evaluate the A/B test results

**What can you tell about the A/A test results?**

In [ ]:
aa = final_df.groupby('group').agg({'user_id':'nunique'})
aa['pct'] = (aa['user_id'] / final_df['user_id'].nunique() * 100).round(2)
aa

The number of users in both groups doesn't vary by more than 1%, as required for a successful A/A test.

In [ ]:

alpha = 0.5 / 5
pvalue = proportions_ztest(
    final_df.query('group == "A"')['user_id'].nunique(), 
    final_df['user_id'].nunique(), value = 0.50)[1]

print('\np-value: {}'.format(pvalue))
if pvalue >= alpha: 
    print('Fail to reject H0: there is not significant difference between groups')
else:
    print('Reject H0: there is a statistically significant difference between the groups.')

**Use the z-criterion to check the statistical difference between the proportions**

In [ ]:
# Create a pivot table to summarize unique user counts for each event by group
pivot = final_df.pivot_table(index='event_name', values='user_id', columns='group', aggfunc='nunique').reset_index()

# Calculate the conversion percentages for groups A and B
pivot['conversion_pct_A'] = ((pivot['A'] / final_df[final_df['group'] == 'A']['user_id'].nunique()) * 100).round(2)
pivot['conversion_pct_B'] = ((pivot['B'] / final_df[final_df['group'] == 'B']['user_id'].nunique()) * 100).round(2)

pivot

H0: There is no statistically significant difference between the groups.

H1: There is a statistically significant difference between the groups.

In [ ]:
def check_hypothesis(group1, group2, event, alpha):
    # Extract the number of users who performed the event in each group
    successes1 = pivot[pivot['event_name'] == event][group1].iloc[0]
    successes2 = pivot[pivot['event_name'] == event][group2].iloc[0]
    
    # Extract the total number of unique users in each group
    trials1 = final_df[final_df['group'] == group1]['user_id'].nunique()
    trials2 = final_df[final_df['group'] == group2]['user_id'].nunique()
    
    # Calculate the proportions of users who performed the event in each group
    p1 = successes1 / trials1
    p2 = successes2 / trials2
    
    # Calculate the combined proportion for both groups
    p_combined = (successes1 + successes2) / (trials1 + trials2)
    
    # Calculate the difference between the two proportions
    difference = p1 - p2
    
    # Calculate the z-value for the difference in proportions
    z_value = difference / math.sqrt(p_combined * (1 - p_combined) * (1 / trials1 + 1 / trials2))
    
    # Get the cumulative distribution function of the standard normal distribution
    distr = st.norm(0, 1)
    
    # Calculate the two-tailed p-value from the z-value
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    # Print the p-value and the conclusion of the hypothesis test
    print(f'Event: {event}, p-value: {p_value}')
    if p_value < alpha:
        print(f'Reject H0 for {event} between groups {group1} and {group2}')
    else:
        print(f'Fail to reject H0 for {event} between groups {group1} and {group2}')


Since we carried out 4 statistical hypothesis tests, with statistical significance level of 0.05., the probability of making at least one Type I error (false positive) increases when making comparisons with the same data.  We'll use the Bonferroni procedure to adjust the significance level (alpaha) to account for the number os tests being perfomed.


In [ ]:
# Loop through each event and test the hypothesis
for event in pivot['event_name'].unique():
    check_hypothesis('A', 'B', event, 0.05)


Since we carried out 4 statistical hypothesis tests, with statistical significance level of 0.05., the probability of making at least one Type I error (false positive) increases when making comparisons with the same data.  We'll use the Bonferroni procedure to adjust the significance level (alpaha) to account for the number os tests being perfomed. We are performing 4 tests and want to maintain an overall significance level of 0.05, the Bonferroni correction sets the significance level for each individual test to 0.05/4=0.0125


In [1]:
bonferroni_alpha = 0.05 / 4
for event in pivot['event_name'].unique():
    check_hypothesis('A', 'B', event, bonferroni_alpha)


NameError: name 'pivot' is not defined

In [ ]:
#iterate over each unique event
for event in pivot['event_name'].unique():
    check_hypothesis('A', 'B', event, bonferroni_alpha)

## 5. Conlusions